In [2]:
import pandas as pd
import numpy as np
import os
import re
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [3]:
# ─────────────────────────────────────────
# 1. Load Data
# ─────────────────────────────────────────
def load_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    print(f"✅ Data loaded: {df.shape[0]} rows")
    print(f"Columns: {list(df.columns)}")
    print(f"\nMedical specialties found ({df['medical_specialty'].nunique()} unique):")
    print(df["medical_specialty"].value_counts().head(15))
    return df

In [4]:
# ─────────────────────────────────────────
# 2. Clean Text
# ─────────────────────────────────────────
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
def preprocess(df: pd.DataFrame, min_samples: int = 100):

    df = df.dropna(subset=["medical_specialty"])

    # Fill missing text columns
    text_columns = ["description", "sample_name", "keywords", "transcription"]
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna("").apply(clean_text)
        else:
            df[col] = ""

    # Keep only specialties with enough samples
    counts = df["medical_specialty"].value_counts()
    valid = counts[counts >= min_samples].index
    df = df[df["medical_specialty"].isin(valid)]

    print(f"\n📊 After filtering: {df.shape[0]} rows")
    print(f"Number of specialties: {df['medical_specialty'].nunique()}")

    # Encode labels
    le = LabelEncoder()
    df["label"] = le.fit_transform(df["medical_specialty"])

    X = df[text_columns]
    y = df["label"]

    return X, y, le

In [6]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

base_model = LinearSVC(
    C=2.0,
    class_weight="balanced"
)

calibrated_svm = CalibratedClassifierCV(
    base_model,
    method="sigmoid",
    cv=3
)

In [7]:
# ─────────────────────────────────────────
# 3. Train Model (Multi-Column TF-IDF)
# ─────────────────────────────────────────
def train(X, y, le):

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )

    # Separate TF-IDF per column
    preprocessor = ColumnTransformer(
        transformers=[
            ("desc", TfidfVectorizer(max_features=8000, ngram_range=(1,2)), "description"),
            ("sample", TfidfVectorizer(max_features=5000, ngram_range=(1,2)), "sample_name"),
            ("keywords", TfidfVectorizer(max_features=5000, ngram_range=(1,2)), "keywords"),
            ("trans", TfidfVectorizer(max_features=20000, ngram_range=(1,2)), "transcription"),
        ],
        remainder="drop"
    )

    pipeline = Pipeline([
        ("features", preprocessor),
        ("clf", calibrated_svm)
    ])

    print("\n⏳ Training model...")
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"\n📈 Overall accuracy: {acc*100:.2f}%")
    print("\n📊 Classification Report:")
    print(classification_report(
        y_test,
        y_pred,
        target_names=le.classes_
    ))

    return pipeline

In [8]:
# ─────────────────────────────────────────
# 4. Save Model
# ─────────────────────────────────────────
def save_model(pipeline, le, output_dir="../models"):

    os.makedirs(output_dir, exist_ok=True)

    # Save model
    joblib.dump(pipeline, os.path.join(output_dir, "specialty_pipeline.pkl"))

    # Save label encoder
    joblib.dump(le, os.path.join(output_dir, "specialty_label_encoder.pkl"))

    # Save class names separately
    classes = list(le.classes_)
    joblib.dump(classes, os.path.join(output_dir, "specialty_classes.pkl"))

    print(f"\n✅ Model artifacts saved to {output_dir}/")
    print(" - specialty_pipeline.pkl")
    print(" - specialty_label_encoder.pkl")
    print(" - specialty_classes.pkl")

    print("\nDetectable specialties:")
    for i, c in enumerate(classes):
        print(f"{i}: {c}")

In [9]:
# ─────────────────────────────────────────
# Main Execution
# ─────────────────────────────────────────
if __name__ == "__main__":
    import sys
    
    csv_path = "D:\\University-Master\\Term-3\\Cloud Computing\\PRO\\Medical-Management-Panel\\data\\mtsamples.csv"

    
    df = load_data(csv_path)
    X, y, le = preprocess(df, min_samples=50)
    pipeline = train(X, y, le)
    save_model(pipeline, le)

✅ Data loaded: 4999 rows
Columns: ['Unnamed: 0', 'description', 'medical_specialty', 'sample_name', 'transcription', 'keywords']

Medical specialties found (40 unique):
medical_specialty
Surgery                          1103
Consult - History and Phy.        516
Cardiovascular / Pulmonary        372
Orthopedic                        355
Radiology                         273
General Medicine                  259
Gastroenterology                  230
Neurology                         223
SOAP / Chart / Progress Notes     166
Obstetrics / Gynecology           160
Urology                           158
Discharge Summary                 108
ENT - Otolaryngology               98
Neurosurgery                       94
Hematology - Oncology              90
Name: count, dtype: int64

📊 After filtering: 4680 rows
Number of specialties: 22

⏳ Training model...

📈 Overall accuracy: 73.08%

📊 Classification Report:
                                precision    recall  f1-score   support

    Cardiovas

In [75]:
def test_bacterial_case(model_path="../models"):

    print("\n🦠 Running bacterial infection test...")

    # Load artifacts
    pipeline = joblib.load(os.path.join(model_path, "specialty_pipeline.pkl"))
    classes = joblib.load(os.path.join(model_path, "specialty_classes.pkl"))

    # Sample bacterial infection case
    test_sample = pd.DataFrame([{
        "description": "Patient with high fever and suspected bacterial infection",
        "sample_name": "Sepsis evaluation",
        "keywords": "fever, infection, bacteria, antibiotics, blood culture",
        "transcription": (
            "The patient presented with high-grade fever and chills. "
            "Blood cultures were obtained and broad-spectrum antibiotics were initiated. "
            "Findings suggest bacterial sepsis requiring inpatient management."
        )
    }])

    # Prediction
    pred_label = pipeline.predict(test_sample)[0]
    pred_class = classes[pred_label]

    print(f"\n🎯 Predicted Specialty: {pred_class}")

    if hasattr(pipeline.named_steps["clf"], "decision_function"):
        probs = pipeline.predict_proba(test_sample)
        confidence = np.max(probs)
        print(f"🔎 Confidence: {confidence:.4f}")

In [77]:
test_bacterial_case()


🦠 Running bacterial infection test...

🎯 Predicted Specialty:  Consult - History and Phy.
